# Math AI Playground

- Evaluate numeric-only math expression step by step.
- Expression length cannot be more than 20 characters.
- Support `*` and `+` only, `-` and `()` support will be attempted later. Intended for the neural network to learn the rules of mathematics.
- Support integers only.

Character encoding:

| Character | Token    |
|-----------|----------|
| 0         | 0        |
| 1         | 1        |
| 2         | 2        |
| 3         | 3        |
| 4         | 4        |
| 5         | 5        |
| 6         | 6        |
| 7         | 7        |
| 8         | 8        |
| 9         | 9        |
| *         | 10       |
| +         | 11       |
| -         | 12       |
|  (space)  | 13       |

In [67]:
# Global imports

from typing import List, Tuple

import random
import re

import pandas as pd

import tensorflow as tf
import tensorflow_text as tf_text
from tensorflow import keras
from tensorflow.keras import layers, losses
import numpy as np

RANDOM_SEED = 1

randomer = random.Random(RANDOM_SEED)

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
CHAR_TOKEN_MAP = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '*': 10,
    '+': 11,
    '-': 12,
    ' ': 13
}

TOKEN_CHAR_MAP = {}

for k, v in CHAR_TOKEN_MAP.items():
    TOKEN_CHAR_MAP[v] = k

TOKEN_CHAR_MAP

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '*',
 11: '+',
 12: '-',
 13: ' '}

In [8]:
CHAR_VOCAB = list(CHAR_TOKEN_MAP)
CHAR_VOCAB

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '+', '-', ' ']

## Data generation

In [9]:
DIGIT_CHARS = list('0123456789')
OP_CHARS = list('*+')
EXPRESSION_LENGTH = 20
TOKEN_SPACE_SIZE = 16


def generate_initial_expression(rand: random.Random) -> str:
    initial_expression = [rand.choice(DIGIT_CHARS) if i % 3 < 2 else rand.choice(OP_CHARS) for i in range(14)]
    if initial_expression[0] == '0':
        initial_expression = initial_expression[1:]
    initial_expression = re.sub("([\*\+\-])0", lambda mo: mo.group(1), ''.join(initial_expression))

    return initial_expression


# Evaluate one step of the expression, return expression, False if no further
# steps can be made anymore.
def progress_expression_step(expression: str) -> Tuple[str, bool]:
    op = ''
    op_idx = -1

    if '*' in expression:
        # Multiplication takes precendence
        op = '*'
        op_idx = expression.find('*')
    else:
        m = re.search('\+', expression)
        if m is not None:
            op = expression[m.start()]
            op_idx = m.start()

    if op == ' ' or op_idx == -1:
        return expression, False

    start_idx = op_idx - 1
    end_idx = op_idx + 1

    while start_idx - 1 >= 0 and expression[start_idx - 1] not in OP_CHARS:
        start_idx -= 1

    while end_idx + 1 < len(expression) and expression[end_idx + 1] not in OP_CHARS:
        end_idx += 1

    num1 = int(expression[start_idx:op_idx])
    num2 = int(expression[op_idx + 1:end_idx + 1])

    calc_result = 0

    if op == '*':
        calc_result = num1 * num2
    elif op == '+':
        calc_result = num1 + num2
    elif op == '-':
        calc_result = num1 - num2

    before_result = expression[:start_idx]
    after_result = expression[end_idx + 1:]

    if len(before_result) > 0 and calc_result < 0:
        if before_result[-1] == '-':
            before_result[-1] = '+'
        elif before_result[-1] == '+':
            before_result[-1] = '-'

    calc_result = abs(calc_result)

    return before_result + str(calc_result) + after_result, True


def generate_expression_with_steps(rand: random.Random) -> List[str]:
    initial_expression = generate_initial_expression(rand)
    ret = [initial_expression]
    while True:
        exp, has_further = progress_expression_step(ret[-1])
        if not has_further:
            break
        ret.append(exp)
    if int(eval(initial_expression)) != int(ret[-1]):
        raise ValueError("internal logic error, value evaluation is incorrect")
    return [l.ljust(20) for l in ret]


generate_expression_with_steps(randomer)

['29*41+77+31+6       ',
 '1189+77+31+6        ',
 '1266+31+6           ',
 '1297+6              ',
 '1303                ']

In [10]:
randomer = random.Random(1)

from_expression_train_file = open('data/from_expression_train.txt', "w")
to_expression_train_file = open('data/to_expression_train.txt', "w")

for i in range(10 ** 6):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps) - 1):
        from_expression_train_file.write(steps[j] + "\n")
        to_expression_train_file.write(steps[j + 1] + "\n")

from_expression_train_file.close()
to_expression_train_file.close()

from_expression_test_file = open('data/from_expression_test.txt', "w")
to_expression_test_file = open('data/to_expression_test.txt', "w")

for i in range(2 * 10 ** 5):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps) - 1):
        from_expression_test_file.write(steps[j] + "\n")
        to_expression_test_file.write(steps[j + 1] + "\n")

from_expression_test_file.close()
to_expression_test_file.close()

from_expression_validation_file = open('data/from_expression_validation.txt', "w")
to_expression_validation_file = open('data/to_expression_validation.txt', "w")

for i in range(2 * 10 ** 5):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps) - 1):
        from_expression_validation_file.write(steps[j] + "\n")
        to_expression_validation_file.write(steps[j + 1] + "\n")

from_expression_validation_file.close()
to_expression_validation_file.close()

In [11]:
% load_ext tensorboard

In [43]:
# Good reference https://www.tensorflow.org/tutorials/load_data/text

PADDED_VOCAB_SIZE = len(CHAR_VOCAB) + 2

keys = [ord(c) for c in CHAR_VOCAB]
values = range(2, len(CHAR_VOCAB) + 2)

init = tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.int64, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

In [44]:
from_expression_train_text_dataset = tf.data.TextLineDataset('data/from_expression_train.txt')
to_expression_train_text_dataset = tf.data.TextLineDataset('data/to_expression_train.txt')
from_expression_validation_text_dataset = tf.data.TextLineDataset('data/from_expression_validation.txt')
to_expression_validation_text_dataset = tf.data.TextLineDataset('data/to_expression_validation.txt')
from_expression_test_text_dataset = tf.data.TextLineDataset('data/from_expression_test.txt')
to_expression_test_text_dataset = tf.data.TextLineDataset('data/to_expression_test.txt')

In [45]:
tokenizer = tf_text.UnicodeCharTokenizer()


def tokenize(text):
    return tokenizer.tokenize(text)


tokenized_example_ds = from_expression_train_text_dataset.map(tokenize)

for text_batch in tokenized_example_ds.take(1):
    print("Tokens: ", text_batch[0])

Tokens:  tf.Tensor(50, shape=(), dtype=int32)


In [83]:
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://stackoverflow.com/questions/41399481/how-do-you-decode-one-hot-labels-in-tensorflow

def preprocess_text(text):
    standardized = tf_text.case_fold_utf8(text)
    tokenized = tokenizer.tokenize(standardized)
    tokenized = tf.cast(tokenized, tf.int64)
    vectorized = vocab_table.lookup(tokenized)
    encoded = tf.one_hot(vectorized, PADDED_VOCAB_SIZE)
    padded = tf.reshape(encoded, (20, 16))
    return padded


example_text = next(iter(from_expression_train_text_dataset))
example_text.numpy()

preprocess_text(example_text)


<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
 

In [84]:
from_expression_train_encoded = from_expression_train_text_dataset.map(preprocess_text)
to_expression_train_encoded = to_expression_train_text_dataset.map(preprocess_text)
from_expression_test_encoded = from_expression_test_text_dataset.map(preprocess_text)
to_expression_test_encoded = to_expression_test_text_dataset.map(preprocess_text)
from_expression_validation_encoded = from_expression_validation_text_dataset.map(preprocess_text)
to_expression_validation_encoded = to_expression_validation_text_dataset.map(preprocess_text)

In [85]:
next(iter(from_expression_train_encoded))

<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
 

In [87]:
train_data = tf.data.Dataset.zip((from_expression_train_encoded, to_expression_train_encoded))
train_data

<ZipDataset shapes: ((20, 16), (20, 16)), types: (tf.float32, tf.float32)>

In [88]:
validation_data = tf.data.Dataset.zip((from_expression_validation_encoded, to_expression_validation_encoded))
validation_data

<ZipDataset shapes: ((20, 16), (20, 16)), types: (tf.float32, tf.float32)>

In [89]:
test_data = tf.data.Dataset.zip((from_expression_test_encoded, to_expression_test_encoded))
test_data

<ZipDataset shapes: ((20, 16), (20, 16)), types: (tf.float32, tf.float32)>

In [90]:
BATCH_SIZE = 32
train_data_batched = train_data.padded_batch(BATCH_SIZE)
validation_data_batched = validation_data.padded_batch(BATCH_SIZE)

In [91]:
next(iter(from_expression_train_encoded))

<tf.Tensor: shape=(20, 16), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
 

In [92]:
def create_model():
    return tf.keras.Sequential([
        layers.Input(shape=(20, 16)),
        layers.Flatten(),
        layers.Dense(20 * 16 * 10),
        layers.Dense(20 * 16 * 10),
        layers.Dense(20 * 16, activation='relu'),
        layers.Reshape((20, 16))
    ])

In [93]:
model = create_model()
model

In [96]:
model.compile(
    optimizer='adam',
    loss=losses.MeanSquaredError(),
    metrics=['accuracy']
)

history = model.fit(train_data, validation_data=validation_data)

ValueError: in user code:

    /home/michaellee8/anaconda3/envs/colab/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/michaellee8/anaconda3/envs/colab/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/michaellee8/anaconda3/envs/colab/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/michaellee8/anaconda3/envs/colab/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/michaellee8/anaconda3/envs/colab/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args